This notebook includes the calculation of volumetric grid charges with a peak price of 67.94 € and the optimization of empirical household energy consumption with flat and dynamic energy prices for this case.

### Requirements

Load the necessary packages to complete the optimization.

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

### Methods

The first method is used in the optimization to determine the start and end points of each charging session, as well as the overall charged energy within one session. The second method is used to write the results of the optimization into a dataframe, enabling further analysis.

In [2]:
#Function to calculate start point, end point, and total energy charged for each charging session
def chargPerUser(load,percent):
    
    #List to store information about charging sessions
    charging_sessions = []
    
    total_energy_charged = 0
    b=False
    b1=False

    #Iterate through all time steps
    for i in range(len(percent)):
        
        #If the plugin hour is not a full hour
        if percent[i] < 1 and percent[i] != 0 and b == False:
            total_energy_charged += load[i] #Add charged energy
            start_index = i #Save the start point
            b = True
            
        #If the plugin hour is a full hour   
        elif percent[i]==1 and percent[i-1]==0:
            total_energy_charged += load[i]
            start_index = i
            b = True
        
        #Full charging hour between plugin and plugout
        elif percent[i] == 1:
            total_energy_charged += load[i]
            b = True
            b1 = True
            
        #Plugout after hours of full charging and the end hour is not a full hour   
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == True and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i #Save the end point
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            b1 = False
            
        #Plugout after no hour of full charging and the end hour is not a full hour 
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == False and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            
        #Plugout in the same hour as plugin
        elif percent[i] == 0 and b == True and b1 == False:
            end_index = start_index
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
        
        #Plugout after hours of full charging and the end hour is a full hour 
        elif percent[i] == 0 and b == True and b1 == True:
            end_index = i-1
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0 
            b = False
            b1 = False
        
        #If no energy is charged or if several not full hours follow each other
        else:
            total_energy_charged += load[i]
            
    return(charging_sessions)

In [3]:
#Write resluts of optimization in dataframe
def get_results_in_df(variableNames, n_timesteps, model):
   

    results_df = pd.DataFrame(columns = variableNames, index = [t for t in range(n_timesteps)] )   #An empty DataFrame is created here, with column entries as the names of the Gurobi variables and rows as the time steps of the decision variables

    for n in variableNames:                                         #Iterate over all target variables
        for t in range(n_timesteps):                                #Iterate over all time steps
            VarName = n + f"[{t}]"                                  #Create a string in the form n[t]. This will be used to read the target variable later
            try:                                                    #If an error occurs in the next line, e.g., due to a typo in my target variables, the function will not stop but continue to execute
                results_df.loc[t][n] =  model.getVarByName(VarName).x    #Read the target variable
            except:
                pass
            
    return results_df

### Dictionaries

The following dictionaries are used to reference column names by indexing:

In [4]:
#Dictionary for households
hh_dic={1:"SFH3_HH",2:"SFH4_HH",3:"SFH5_HH",4:"SFH7_HH",5:"SFH9_HH",6:"SFH12_HH",7:"SFH14_HH",8:"SFH16_HH",9:"SFH18_HH",10:"SFH19_HH",
        11:"SFH20_HH",12:"SFH21_HH",13:"SFH22_HH",14:"SFH27_HH",15:"SFH28_HH",16:"SFH29_HH",17:"SFH30_HH",18:"SFH32_HH",19:"SFH34_HH",
        20:"SFH36_HH",21:"SFH38_HH",22:"SFH39_HH"}

In [5]:
#Dictionary for heatpumps
hp_dic={1:"SFH3_HP",2:"SFH4_HP",3:"SFH5_HP",4:"SFH7_HP",5:"SFH9_HP",6:"SFH12_HP",7:"SFH14_HP",8:"SFH16_HP",9:"SFH18_HP",10:"SFH19_HP",
        11:"SFH20_HP",12:"SFH21_HP",13:"SFH22_HP",14:"SFH27_HP",15:"SFH28_HP",16:"SFH29_HP",17:"SFH30_HP",18:"SFH32_HP",19:"SFH34_HP",
        20:"SFH36_HP",21:"SFH38_HP",22:"SFH39_HP"}

### Optimization

The following method describes the optimization of empirical household consumption with volumetric grid charges and additional peak price grid charges to minimize total costs. HH consumption remains inflexible. Shifting HP consumption is restricted by a maximum number of blocked time steps (sigma_max) in each interval of length 'length'. The total consumption over an interval must match the empirical consumption within that interval. EV shifting is freely possible between plug-in and plug-out times, as long as the maximum charging power (ev_max) of the wallbox is not surpassed and the charged energy for the session matches the empirical data.

In [6]:
#Optimization of one household with hp flexibility within intervals and flexible ev between plugin and plugout without dynamic energy prices
def optimized_HP_EV_flat(hh, hp, ev_user, peak_ev, flexibility, length, p_flex, p_grid, p_peak):
    
    l = len(hh)
    l_ev= len(ev_user.columns)//2
    ev_max = peak_ev
    hp_max = max(hp)
    sigma_max = flexibility
    part = length
    
    #1. create Model
    model  = Model("simpleHP")
        

    #2. define Variables
    
    #2.1 hp load over all time steps
    d_hp = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, ub=hp_max, name="d_hp")
    
    #2.2 procentual hp shifts over all time steps
    sigma = model.addVars(l,vtype=GRB.BINARY, name="sigma")
    
    #2.3 ev charging load over all time steps
    d_ev = model.addVars(l, l_ev, vtype=GRB.CONTINUOUS, lb=0, ub=ev_max, name="d_ev")
    
    #2.4 total consumption over all time steps
    d = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="d")

    #2.5 peak demand over time period
    max_net_energy = model.addVar(name="max_net_energy")
    

    #3. constraints
    
    #3.1 total consumption over all time steps as sum of hp, hh and all evs
    model.addConstrs(d[t] == d_hp[t]+hh[t]+quicksum(d_ev[t,u] for u in range(l_ev)) for t in range(l))
    
    #HP
    #3.2 calculation of hp load
    model.addConstrs(d_hp[t] >= (1-sigma[t])*hp[t] for t in range(l))
    
    #3.3 hp flexibility is restricted to flex_hours through binary variable sigma
    model.addConstrs(quicksum(sigma[k] for k in range(part*i,part*i+part)) <= sigma_max for i in range(l//part))

    #3.4 hp load over time part must at least be empirical hp
    model.addConstrs(quicksum(d_hp[k] for k in range(part*i,part*i+part)) >= quicksum(hp[k] for k in range(part*i,part*i+part)) for i in range(l//part))

    
    #EV
    #iterate through all evs (users) that charge at this garage
    for u in range(l_ev):
        ev = ev_user.iloc[:,u*2]
        ev_perc = ev_user.iloc[:,u*2+1]
        
        #3.5 in every timestep ev power demand must be smaller than ev_max
        model.addConstrs(d_ev[t,u] <= ev_perc[t]*ev_max for t in range(l))
        
        #get for every charging session start and end time and the total charged energy
        charging_sessions=chargPerUser(ev,ev_perc)
        list_length = len(charging_sessions)
        
        #iterate through all charging sessions of one user
        for i in range (list_length):
            start_index = charging_sessions[i]['start_index']
            end_index = charging_sessions[i]['end_index']
            total_energy = charging_sessions[i]['total_energy']
            
            #3.6 the charged power between start and end must be equal to the total charged energy
            model.addConstr(quicksum(d_ev[t,u] for t in range(start_index,end_index+1)) == total_energy)

    #determine peak
    for t in range(l):
        model.addConstr(max_net_energy >= d[t], name=f"max_constraint_{t}")

            
    #4. optimize
    objective_expr = quicksum(d[t] * (p_flex+p_grid) for t in range(l))#(p_flex.iloc[t,0]+p_grid) for t in range(l))
    
    #4.1 set the costs as the variable to be minimized
    model.setObjective(objective_expr+max_net_energy*p_peak,GRB.MINIMIZE)

    #4.2 perform optimization
    model.optimize()
    
    #5 return results
    #5.1 get objective value
    cost = model.getObjective().getValue()
    
    #get values for total load d for every hour
    var =["d"]
    load_after_shift = get_results_in_df(var,l, model)
    
    return cost, load_after_shift

In [7]:
#Optimization of one household with hp flexibility within intervals and flexible ev between plugin and plugout with dynamic energy prices
def optimized_HP_EV(hh, hp, ev_user, peak_ev, flexibility, length, p_flex, p_grid, p_peak):
    
    l = len(hh)
    l_ev= len(ev_user.columns)//2
    ev_max = peak_ev
    hp_max = max(hp)
    sigma_max = flexibility
    part = length
    
    #1. create Model
    model  = Model("simpleHP")
        

    #2. define Variables
    
    #2.1 hp load over all time steps
    d_hp = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, ub=hp_max, name="d_hp")
    
    #2.2 procentual hp shifts over all time steps
    sigma = model.addVars(l,vtype=GRB.BINARY, name="sigma")
    
    #2.3 ev charging load over all time steps
    d_ev = model.addVars(l, l_ev, vtype=GRB.CONTINUOUS, lb=0, ub=ev_max, name="d_ev")
    
    #2.4 total consumption over all time steps
    d = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="d")

    #2.5 peak demand over time period
    max_net_energy = model.addVar(name="max_net_energy")
    

    #3. constraints
    
    #3.1 total consumption over all time steps as sum of hp, hh and all evs
    model.addConstrs(d[t] == d_hp[t]+hh[t]+quicksum(d_ev[t,u] for u in range(l_ev)) for t in range(l))
    
    #HP
    #3.2 calculation of hp load
    model.addConstrs(d_hp[t] >= (1-sigma[t])*hp[t] for t in range(l))
    
    #3.3 hp flexibility is restricted to flex_hours through binary variable sigma
    model.addConstrs(quicksum(sigma[k] for k in range(part*i,part*i+part)) <= sigma_max for i in range(l//part))

    #3.4 hp load over time part must at least be empirical hp
    model.addConstrs(quicksum(d_hp[k] for k in range(part*i,part*i+part)) >= quicksum(hp[k] for k in range(part*i,part*i+part)) for i in range(l//part))

    
    #EV
    #iterate through all evs (users) that charge at this garage
    for u in range(l_ev):
        ev = ev_user.iloc[:,u*2]
        ev_perc = ev_user.iloc[:,u*2+1]
        
        #3.5 in every timestep ev power demand must be smaller than ev_max
        model.addConstrs(d_ev[t,u] <= ev_perc[t]*ev_max for t in range(l))
        
        #get for every charging session start and end time and the total charged energy
        charging_sessions=chargPerUser(ev,ev_perc)
        list_length = len(charging_sessions)
        
        #iterate through all charging sessions of one user
        for i in range (list_length):
            start_index = charging_sessions[i]['start_index']
            end_index = charging_sessions[i]['end_index']
            total_energy = charging_sessions[i]['total_energy']
            
            #3.6 the charged power between start and end must be equal to the total charged energy
            model.addConstr(quicksum(d_ev[t,u] for t in range(start_index,end_index+1)) == total_energy)

    #determine peak
    for t in range(l):
        model.addConstr(max_net_energy >= d[t], name=f"max_constraint_{t}")

            
    #4. optimize
    objective_expr = quicksum(d[t] * (p_flex.iloc[t,0]+p_grid) for t in range(l))
    
    #4.1 set the costs as the variable to be minimized
    model.setObjective(objective_expr+max_net_energy*p_peak,GRB.MINIMIZE)

    #4.2 perform optimization
    model.optimize()
    
    #5 return results
    #5.1 get objective value
    cost = model.getObjective().getValue()
    
    #get values for total load d for every hour
    var =["d"]
    load_after_shift = get_results_in_df(var,l, model)
    
    return cost, load_after_shift

### Read in data

Read in EV charging data, household and heat pump consumption patterns, day-ahead market prices, garage IDs, total load in the empirical case, as well as the costs in the empirical scenario. All data has been pre-processed beforehand.

In [8]:
ev_hourly = pd.read_csv('Hourly_EV_Charging.csv') #Charging data
hh_hp = pd.read_csv('Hourly_HH_HP.csv') #Hourly HP and HH consumption patterns
peak_ev = pd.read_csv('Min_Peak_Load.csv') #Maximum charging power of each charging station
p_flex = pd.read_csv('Hourly_Flexible_Prices.csv') #Day-ahead market prices
unique_garage = pd.read_csv('Unique_Garage.csv') #Garage IDs
load_before_shift = pd.read_csv('load_before_shift.csv',index_col=0) #empirical total load
cost_before_shift_flat = pd.read_csv('cost_before_shift.csv',index_col=0) #costs with volumetric grid charges and empirical consumption

In [9]:
p_grid_alt=0.2695 #grid charges per kWh in case of only volumetric grid charges
p_peak = 67.94 #prices for highest peak over the observation period

Calculate costs before shift with peak pricing to determine the volumetric grid charges in the case of addtional peak pricing

In [10]:
#Calculate costs with empirical data, the grid charges of the volumetric case (used in the scenario before) and the additional peak price
cost_before_shift_peak = []

for i in range(len(peak_ev)): #Iterate over all HHs and determine total costs for each HH with empirical consumption patterns
    
    cost = 0
    max_load= load_before_shift.iloc[:,i].max()
    
    for k in range(len(p_flex)):
        cost += load_before_shift.iloc[k,i]*(p_flex.iloc[k,0]+p_grid_alt)
        
    cost_before_shift_peak.append(cost+p_peak*max_load)

In [11]:
#Calculate costs per kWh first in the case of only volumetric grid charges and then in the case of volumetric grid charges with additional peak pricing
c_per_kWh_flat = (cost_before_shift_flat.sum()/load_before_shift.sum().sum()).item()
c_per_kWh_peak=(sum(cost_before_shift_peak)/load_before_shift.sum().sum()).item()
#determine the difference between the two cases and with it the new grid charges in case of additional peak pricing
diff= c_per_kWh_flat-c_per_kWh_peak
p_grid = p_grid_alt+diff

Calculate the energy costs per kWh in case of flat energy tariff in combination with volumetric grid charges with additional peak pricing

In [13]:
cost_before_shift_peak_p_flat = []

for i in range(len(peak_ev)): #Iterate over all HHs and calculate total costs with empirical consumption patterns
    cost = 0
    max_load= load_before_shift.iloc[:,i].max()
    for k in range(len(p_flex)):
        cost += load_before_shift.iloc[k,i]*(p_grid)
    cost_before_shift_peak_p_flat.append(cost+p_peak*max_load)

In [14]:
p_flat = (sum(cost_before_shift_flat['cost_before_shift'].tolist())-sum(cost_before_shift_peak_p_flat))/load_before_shift.sum().sum()

### Perform optimization

Perform optimization for each HH, first for flat energy prices and then for dynamic energy prices, to determine the new household consumption patterns for each scenario.

In [15]:
load_after_shift_p_flat = pd.DataFrame()
cost_after_shift_p_flat = []
#p_flat = 0.038977773713860416

#Iterate over all households
for i in range(len(peak_ev)):
    
    #Select all users per garage
    selected_columns = ev_hourly.filter(regex=f'^{unique_garage.iloc[i,0]}-', axis=1)
    
    #Perform optimization
    cost, load = optimized_HP_EV_flat(hh_hp[hh_dic[i+1]],hh_hp[hp_dic[i+1]], selected_columns, peak_ev.iloc[i,0],2,4, p_flat, p_grid,p_peak)
    
    #Save total costs
    cost_after_shift_p_flat.append(cost)
    
    #Store shifted loads per household
    load_after_shift_p_flat[f"{hh_dic[i+1]}_d"] = load.iloc[:,0]

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-31
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 62069 rows, 57025 columns and 137826 nonzeros
Model fingerprint: 0x1eeef6e8
Variable types: 47521 continuous, 9504 integer (9504 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+00]
  Objective range  [3e-01, 7e+01]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e-02, 4e+01]
Found heuristic solution: objective 3827.7669799
Presolve removed 53571 rows and 46075 columns
Presolve time: 0.38s
Presolved: 8498 rows, 10950 columns, 28366 nonzeros
Found heuristic solution: objective 3811.4743596
Variable types: 7446 continuous, 3504 integer (3504 binary)

Root relaxation: objective 2.817590e+03, 4455 iteratio

In [16]:
load_after_shift = pd.DataFrame()
cost_after_shift = []

#Iterate over all households
for i in range(len(peak_ev)):
    
    #Select all users per garage
    selected_columns = ev_hourly.filter(regex=f'^{unique_garage.iloc[i,0]}-', axis=1)
    
    #Perform optimization
    cost, load = optimized_HP_EV(hh_hp[hh_dic[i+1]],hh_hp[hp_dic[i+1]], selected_columns, peak_ev.iloc[i,0],2,4, p_flex, p_grid,p_peak)
    
    #Save total costs
    cost_after_shift.append(cost)
    
    #Store shifted loads per household
    load_after_shift[f"{hh_dic[i+1]}_d"] = load.iloc[:,0]

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 62069 rows, 57025 columns and 137826 nonzeros
Model fingerprint: 0x679835b4
Variable types: 47521 continuous, 9504 integer (9504 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+00]
  Objective range  [1e-01, 7e+01]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e-02, 4e+01]
Found heuristic solution: objective 3810.2779659
Presolve removed 52865 rows and 45079 columns
Presolve time: 0.87s
Presolved: 9204 rows, 11946 columns, 30783 nonzeros
Found heuristic solution: objective 3787.3003959
Variable types: 8126 continuous, 3820 integer (3820 binary)

Root relaxation: objective 2.750756e+03, 5773 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bound

### Save Data

In [19]:
#save as a dataframe
cost_after_shift_p_flat_df = pd.DataFrame(cost_after_shift_p_flat, columns=['cost_before_shift'])
cost_after_shift_df = pd.DataFrame(cost_after_shift, columns=['cost_after_shift'])

cost_after_shift_p_flat_df.to_csv('cost_after_shift_peak_flat.csv')
cost_after_shift_df.to_csv('cost_after_shift_peak_dyn.csv')
load_after_shift_p_flat.to_csv('load_after_shift_peak_flat.csv')
load_after_shift.to_csv('load_after_shift_peak_dyn.csv')